In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

### load the crsp data

数据进行按照日历年,进行年化处理

In [4]:
crsp = pd.read_csv('data/return_ensembled.zip',
                   parse_dates=['date'], infer_datetime_format=True)
crsp.date = pd.to_datetime(crsp.date)
crsp = crsp.set_index('date')

In [5]:
crsp.head()

,permno,mv,mv_adj,ret_p1,retx_p1,dret_p1
date,,,,,,
1973-01-31,63845,106005.375,526734.998484,0.724138,0.724138,0.945299
1973-01-31,15580,4425.000,21987.586651,1.043478,1.043478,0.999527
1973-01-31,28820,6768.750,33633.554157,1.187500,1.187500,0.988575
1973-01-31,11253,3038.000,15095.658361,0.942308,0.942308,0.999527
1973-01-31,61903,766.500,3808.697213,0.954545,0.954545,1.015606


In [6]:
def calendar_resample(df):
    res = df.resample('Y').apply({'ret_p1':'cumprod', 'retx_p1':'cumprod',
                                 'dret_p1':'cumprod'})
    res = pd.concat([df.mv, df.mv_adj, res], axis=1)
    res = res.resample('Y').last()
    
#     ret_p1 = df.ret_p1.cumprod().iloc[-1]
#     retx_p1 = df.retx_p1.cumprod().iloc[-1]
#     mv = df.mv.resample('Y').last().iloc[-1]
#     mv_adj = df.mv_adj.resample('Y').last().iloc[-1]
#     res = pd.DataFrame({'ret_p1':ret_p1, 'retx_p1':retx_p1,
#                         'mv':mv, 'mv_adj':mv_adj}, index=[df.index[-1]])
    return res

In [13]:
# crsp[crsp.permno==77418].groupby('permno').apply(calendar_resample)

In [9]:
tic = time.perf_counter()
crsp_ca = crsp.groupby('permno').apply(calendar_resample)
toc = time.perf_counter()
print(toc-tic)
crsp_ca.reset_index().to_csv('data/crsp_ca.csv', index=False)

215.8994486


In [11]:
crsp_ca = pd.read_csv('data/crsp_ca.csv',
                     parse_dates=['date'], infer_datetime_format=True)
# crsp_ca = crsp_ca.reset_index()

查看firm-years数量

ps:因为share outstanding的单位默认是1000, 故mv_adj只需要大于50000即可

In [12]:
crsp_ca[crsp_ca.mv_adj > 5e+4].shape[0]

160224

In [20]:
crsp_ca = crsp_ca[crsp_ca.mv_adj > 5e+4]

In [7]:
def calc_exf(df):
    exf = df.mv - df.mv.shift(1) * df.retx_p1
    res = pd.DataFrame({
        'date':df.date,
        'mv':df.mv,
        'mv_adj':df.mv_adj,
        'ret_p1':df.ret_p1,
        'retx_p1':df.retx_p1,
        'dret_p1':df.dret_p1,
        'exf':exf
    })
    return res

In [15]:
tic = time.perf_counter()
crsp_ca = crsp_ca.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

39.44277480000005


In [87]:
# crsp_ca.query('permno == 77418')

In [14]:
sample = crsp_ca.query('permno == 77418').set_index('permno').groupby('permno').apply(calc_exf)
sample

,date,mv,mv_adj,ret_p1,retx_p1,dret_p1,exf
permno,,,,,,,
77418,1992-12-31,1.678072e+05,2.492911e+05,1.918033,1.918033,1.080662,NaN
77418,1993-12-31,4.111965e+05,5.941635e+05,2.000764,2.000001,1.170278,7.558184e+04
77418,1994-12-31,9.291520e+05,1.308600e+06,1.914528,1.914528,0.970260,1.419047e+05
77418,1995-12-31,3.246112e+06,4.458880e+06,2.678568,2.678568,1.260227,7.573156e+05
77418,1996-12-31,3.104486e+06,4.124966e+06,0.886665,0.886665,1.222912,2.262706e+05
77418,1997-12-31,9.313355e+06,1.216826e+07,2.721810,2.721810,1.322655,8.635347e+05
77418,1998-12-31,7.106990e+07,9.138707e+07,6.934765,6.934712,1.271768,6.484468e+06
77418,1999-12-31,1.696175e+08,2.124218e+08,1.956485,1.956485,1.232681,3.057033e+07
77418,2000-12-31,8.087910e+07,9.792485e+07,0.458649,0.458649,0.886900,3.084190e+06


,date,mv,mv_adj,ret_p1,retx_p1,dret_p1,exf
168264,1992-12-31,1.678072e+05,2.492911e+05,1.918033,1.918033,1.080662,NaN
168265,1993-12-31,4.111965e+05,5.941635e+05,2.000764,2.000001,1.170278,7.558184e+04
168266,1994-12-31,9.291520e+05,1.308600e+06,1.914528,1.914528,0.970260,1.419047e+05
168267,1995-12-31,3.246112e+06,4.458880e+06,2.678568,2.678568,1.260227,7.573156e+05
168268,1996-12-31,3.104486e+06,4.124966e+06,0.886665,0.886665,1.222912,2.262706e+05
168269,1997-12-31,9.313355e+06,1.216826e+07,2.721810,2.721810,1.322655,8.635347e+05
168270,1998-12-31,7.106990e+07,9.138707e+07,6.934765,6.934712,1.271768,6.484468e+06
168271,1999-12-31,1.696175e+08,2.124218e+08,1.956485,1.956485,1.232681,3.057033e+07
168272,2000-12-31,8.087910e+07,9.792485e+07,0.458649,0.458649,0.886900,3.084190e+06
168273,2001-12-31,1.365999e+08,1.627787e+08,0.922413,0.922413,0.863769,6.199592e+07


In [77]:
# writer = pd.ExcelWriter('data/example.xlsx', datetime_format='YYYY-MM-DD')
# sample.to_excel(writer, sheet_name='sample', index=False)
# writer.save()

In [90]:
def calc_wrt(df):
    ret = df.ret_p1
    dret = df.dret_p1
    exf = df.exf
    exf_to_mv = exf / df.mv.shift(1)
    
    car1 = ret[::-1].rolling(1, min_periods=0).apply(np.prod, raw=True)[::-1] - \
        dret[::-1].rolling(1, min_periods=0).apply(np.prod, raw=True)[::-1]
    car3 = ret[::-1].rolling(3, min_periods=0).apply(np.prod, raw=True)[::-1] - \
        dret[::-1].rolling(3, min_periods=0).apply(np.prod, raw=True)[::-1]
    car5 = ret[::-1].rolling(5, min_periods=0).apply(np.prod, raw=True)[::-1] - \
        dret[::-1].rolling(5, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    return pd.DataFrame({
        'date':df.date,
        'exf_to_mv':exf_to_mv,
        # 这里做了时间平移处理, 数据对齐，详见后面sample说明
        'car1':car1.shift(-1),
        'car3':car3.shift(-1),
        'car5':car5.shift(-1)
    })

In [89]:
def calc_wrt_full(df):
    ret = df.ret_p1
    dret = df.dret_p1
    exf = df.exf
    exf_to_mv = exf / df.mv.shift(1)
    
    ret_n1 = ret[::-1].rolling(1, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    ret_n2 = ret[::-1].rolling(2, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    ret_n3 = ret[::-1].rolling(3, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    ret_n4 = ret[::-1].rolling(4, min_periods=0).apply(np.prod, raw=True)[::-1]
        
    ret_n5 = ret[::-1].rolling(5, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    ret_n6 = ret[::-1].rolling(6, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    ret_n7 = ret[::-1].rolling(7, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    ret_n8 = ret[::-1].rolling(8, min_periods=0).apply(np.prod, raw=True)[::-1]
    
    return pd.DataFrame({
        'date':df.date,
        'exf_to_mv':exf_to_mv,
        # 这里做了时间平移处理, 数据对齐，详见后面sample说明
        'ret_n1':ret_n1.shift(-1),
        'ret_n2':ret_n2.shift(-1),
        'ret_n3':ret_n3.shift(-1),
        'ret_n4':ret_n4.shift(-1),
        'ret_n5':ret_n5.shift(-1),
        'ret_n6':ret_n6.shift(-1),
        'ret_n7':ret_n7.shift(-1),
        'ret_n8':ret_n8.shift(-1)
    })

这里做了平移之后，比如我想看2001年的WR5,直接看2001年对应的wr5列的数据即可

In [46]:
sample.groupby('permno').apply(calc_wrt_full).head()

,date,exf_to_mv,wr1,wr2,wr3,wr4,wr5,wr6,wr7,wr8
permno,,,,,,,,,,
77418,1992-12-31,NaN,1.709649,3.373498,7.170250,5.198750,10.698182,58.335606,92.588998,47.881219
77418,1993-12-31,0.450409,1.973211,4.193990,3.040829,6.257533,34.121402,54.156744,28.006469,29.907941
77418,1994-12-31,0.345102,2.125464,1.541056,3.171243,17.292319,27.445991,14.193344,15.156987,7.885124
77418,1995-12-31,0.815061,0.725044,1.492024,8.135784,12.912939,6.677762,7.131142,3.709836,3.920974
77418,1996-12-31,0.069705,2.057838,11.221084,17.809859,9.210141,9.835455,5.116702,5.407909,5.236322


In [92]:
tic = time.perf_counter()
data_wr = crsp_ca.groupby('permno').apply(calc_wrt)

data_wr = data_wr.reset_index()
data_wr1 = data_wr[data_wr.exf_to_mv.notna()]

toc = time.perf_counter()
print(toc-tic)

122.23827340000025


In [95]:
percet

[0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9]

In [103]:
bins = data_wr.describe(percentiles=percet).exf_to_mv.iloc[4:].tolist()
data_wr.groupby(np.digitize(data_wr.exf_to_mv,bins)).mean().drop('permno', axis=1)

,exf_to_mv,wr1,wr3,wr5
0,-0.118307,0.083823,0.242858,0.395156
1,-0.011738,0.075604,0.197148,0.316363
2,-0.000171,0.067554,0.206035,0.319339
3,0.000178,0.069257,0.194958,0.358279
4,0.002567,0.083295,0.240115,0.375907
5,0.007419,0.081822,0.220006,0.350367
6,0.016538,0.076665,0.228949,0.353202
7,0.038504,0.078596,0.210068,0.295869
8,0.111294,0.041842,0.132808,0.205709
9,0.673833,0.020471,0.034599,0.076031


In [93]:
tic = time.perf_counter()
data_wr_full = crsp_ca.groupby('permno').apply(calc_wrt_full)

data_wr_full = data_wr_full.reset_index()
data_wr_full = data_wr_full[data_wr_full.exf_to_mv.notna()]

toc = time.perf_counter()
print(toc-tic)

157.73844760000065


In [91]:
data_wr_full.head()

,permno,date,exf_to_mv,wr1,wr2,wr3,wr4,wr5,wr6,wr7,wr8
1,10002,1994-12-31,-2.221426e-07,0.795148,0.680830,1.062996,1.073338,0.640625,0.547393,0.489801,0.639906
2,10002,1995-12-31,1.065433e-03,0.856230,1.336853,1.349860,0.805667,0.688416,0.615988,0.804763,0.721246
3,10002,1996-12-31,-4.298500e-08,1.561324,1.576516,0.940947,0.804009,0.719418,0.939891,0.842351,1.120017
4,10002,1997-12-31,7.809677e-01,1.009730,0.602660,0.514953,0.460774,0.601983,0.539511,0.717350,0.577257
5,10002,1998-12-31,1.993716e-01,0.596853,0.509991,0.456334,0.596182,0.534312,0.710438,0.571695,0.619460


In [94]:
# data_wr_full.groupby(pd.cut(data_wr_full.exf_to_mv, 10)).agg(
#         {
#             'exf_to_mv': 'count',
#             'wr1': np.mean,
#             'wr2': np.mean,
#             'wr3': np.mean,
#             'wr4': np.mean,
#             'wr5': np.mean,
#             'wr6': np.mean,
#             'wr7': np.mean,
#             'wr8': np.mean
#         }
#     )

In [98]:
data_wr_full.head()

,permno,date,exf_to_mv,wr1,wr2,wr3,wr4,wr5,wr6,wr7,wr8
1,10002,1994-12-31,-2.221426e-07,1.092129,1.043990,2.049311,1.950594,1.600282,1.260346,1.411059,1.653241
2,10002,1995-12-31,1.065433e-03,0.955922,1.876438,1.786048,1.465287,1.154027,1.292026,1.513779,2.259641
3,10002,1996-12-31,-4.298500e-08,1.962960,1.868403,1.532852,1.207239,1.351602,1.583579,2.363833,3.730554
4,10002,1997-12-31,7.809677e-01,0.951829,0.780888,0.615009,0.688553,0.806730,1.204218,1.900473,1.593407
5,10002,1998-12-31,1.993716e-01,0.820407,0.646134,0.723400,0.847558,1.265162,1.996654,1.674047,2.170798


In [101]:
# groups = df.groupby(np.digitize(df.a,bins))
bins = data_wr_full.describe(percentiles=percet).exf_to_mv.iloc[4:].tolist()
data_wr_full.groupby(np.digitize(data_wr_full.exf_to_mv,bins)).agg(
        {
            'exf_to_mv': np.mean,
            'wr1': np.mean,
            'wr2': np.mean,
            'wr3': np.mean,
            'wr4': np.mean,
            'wr5': np.mean,
            'wr6': np.mean,
            'wr7': np.mean,
            'wr8': np.mean
        }
    )

,exf_to_mv,wr1,wr2,wr3,wr4,wr5,wr6,wr7,wr8
0,-0.118307,1.204962,1.387208,1.591313,1.779226,1.980229,2.164837,2.367988,2.493689
1,-0.011738,1.212370,1.390668,1.572539,1.751308,1.954990,2.121454,2.295055,2.473855
2,-0.000171,1.252262,1.495934,1.729828,1.971600,2.234998,2.459182,2.709223,2.992397
3,0.000178,1.247806,1.485923,1.705331,1.974950,2.258721,2.516442,2.774328,3.026113
4,0.002567,1.239576,1.475992,1.682353,1.900553,2.128776,2.353939,2.572026,2.830561
5,0.007419,1.227575,1.439582,1.633823,1.822304,2.039783,2.209463,2.384945,2.556944
6,0.016538,1.213211,1.424883,1.613662,1.790218,1.980065,2.137753,2.281505,2.436769
7,0.038504,1.207332,1.417959,1.584804,1.754580,1.900602,2.048666,2.201186,2.364626
8,0.111294,1.177650,1.359000,1.530323,1.684945,1.855929,1.985534,2.152014,2.326789
9,0.673833,1.136086,1.266623,1.387572,1.527644,1.637577,1.761673,1.866787,1.983282


In [ ]:
pd.Series.count()

In [64]:
res1 = data_wr_full.groupby(pd.cut(data_wr_full.exf_to_mv, 10)).\
mean().drop('permno', axis=1)

In [66]:
res1.columns = ['exf_to_mv_mean', 'wr1', 'wr2', 'wr3', 'wr4', 'wr5', 'wr6', 'wr7', 'wr8']

In [67]:
res1 = res1.reset_index()

In [70]:
res1.loc[0]

exf_to_mv         (-3.693, 3.5]
exf_to_mv_mean        0.0607879
wr1                      1.0616
wr2                     1.11093
wr3                      1.1458
wr4                     1.17018
wr5                     1.19277
wr6                     1.21112
wr7                     1.22719
wr8                     1.23888
Name: 0, dtype: object